<a href="https://colab.research.google.com/github/Christianatgithub/AgenticAI/blob/main/DictionarySwap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install --upgrade litellm

import sys, importlib.metadata

print("Python:", sys.version.split()[0])
print("LiteLLM:", importlib.metadata.version("litellm"))


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.4/278.4 kB 23.0 MB/s eta 0:00:00
Python: 3.12.11
LiteLLM: 1.77.1


In [3]:
from google.colab import userdata
PROVIDER = "gemini"  # or "groq" or "openai"

if PROVIDER == "gemini":
    api_key = userdata.get("GEMINI_API_KEY")
    model = "gemini/gemini-1.5-flash"
elif PROVIDER == "groq":
    api_key = userdata.get("GROQ_API_KEY")
    model = "groq/llama-3.1-8b-instant"
elif PROVIDER == "openai":
    api_key = userdata.get("OPENAI_API_KEY")
    model = "gpt-4o-mini"

def chat(msg):
    from litellm import completion
    return completion(
        model=model,
        messages=[{"role":"user","content":msg}],
        api_key=api_key
    ).choices[0].message["content"]

In [5]:
from litellm import completion
from typing import List, Dict
from google.colab import userdata

api_key = userdata.get("GEMINI_API_KEY")

def generate_response(messages: List[Dict], api_key: str) -> str:
    """Call LLM to get response"""
    response = completion(
        model="gemini/gemini-1.5-flash",
        messages=messages,
        max_tokens=1024,
        api_key=api_key
    )
    return response.choices[0].message.content


messages = [
    {"role": "system", "content": "You are an expert software engineer that prefers functional programming."},
    {"role": "user", "content": "Write a function to swap the keys and values in a dictionary."}
]

response = generate_response(messages, api_key)
print(response)

The straightforward approach to swapping keys and values in a dictionary might seem to be a simple loop.  However, this has several potential problems:

1. **Duplicate Values:** If the original dictionary has duplicate values, the resulting swapped dictionary will lose data because keys must be unique.  A simple loop won't handle this gracefully.
2. **Immutability:**  Modifying dictionaries in place can lead to unexpected side effects and makes the code harder to reason about, especially in concurrent environments.  A functional approach promotes immutability.
3. **Error Handling:**  What happens if the values aren't hashable (e.g., lists or other mutable objects)?  Robust code should handle this gracefully.


Here's a Python function that addresses these concerns using a functional approach, leveraging the power of `dict.items()` and `zip`, and handles potential errors:


```python
from typing import Dict, Any, Tuple, Union

def swap_dict(input_dict: Dict[Any, Any]) -> Union[Dict[Any,